In [1]:
from src.resnet_main import *

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [3]:
config = {"output_path" : "/home/jl5307/current_research/AMD_prediction/results/resnet101_amd_detection/",
         "data_root_path" : "/home/jl5307/current_research/AMD_prediction/img_data/img_files/",
         "data_dict_path" : "/home/jl5307/current_research/AMD_prediction/img_data/5_fold/five_fold_amd_detection_data_dict.pkl", 
          "architecture" : [3,4,23,3], "epoch" : 20, "batch_size" : 32, "learning_rate" : 0.0005, "l2_reg" : 0.001, "num_class" : 1, 
          "resizing" : 256, "scale" : 0.875, "class_weight" : [0.13, 0.87], "prediction" : "binary", "augment" : True, 
          "lr_scheduling" : [2000, 0.9], "specify_fold" : 1, "use_pretrain" : "imagenet"}

In [4]:
print("load data...")
data_dict = load_data("/home/jl5307/current_research/AMD_prediction/img_data/5_fold/five_fold_amd_detection_data_dict.pkl")
data_dict = data_dict[1]

load data...


In [5]:
training_set_dict = data_dict["train_set"]
validation_set_dict = data_dict["validation_set"]
test_set_dict = data_dict["test_set"]

In [6]:
        print("use ResNet pretrained with imagenet...")
        resnet = ResNetPretrained(config)

use ResNet pretrained with imagenet...


In [7]:
training_set = list(training_set_dict.keys())

In [8]:
i = 1
batch_size = 32

In [9]:
AUGMENTER = set_augmenter()

In [10]:
eye_batch = training_set[i*batch_size:(i+1)*batch_size]
x, y = prepare_batch(eye_batch, training_set_dict, config, augmenter=AUGMENTER, one_hot=False)

In [11]:
y_hat = resnet(x, training=True)

In [12]:
y_hat

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.3500385 ],
       [0.5636182 ],
       [0.42013985],
       [0.3185327 ],
       [0.31532097],
       [0.23526669],
       [0.34857997],
       [0.42293125],
       [0.34598666],
       [0.2712548 ],
       [0.29288244],
       [0.2212107 ],
       [0.5063108 ],
       [0.3341339 ],
       [0.36088514],
       [0.4881481 ],
       [0.60613424],
       [0.2472348 ],
       [0.583323  ],
       [0.22052313],
       [0.38801134],
       [0.46745464],
       [0.4588426 ],
       [0.51046157],
       [0.17800114],
       [0.2724768 ],
       [0.2828884 ],
       [0.36663193],
       [0.46094686],
       [0.50215006],
       [0.28557032],
       [0.47338086]], dtype=float32)>

In [14]:
y

<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

In [17]:
y_hat_flatten = tf.reshape(y_hat, shape=-1)
y_flatten = tf.reshape(y, shape=-1)

In [24]:
class WeightedBinaryCrossEntropy(tf.keras.losses.Loss):
    def __init__(self, class_weight):
        super(WeightedBinaryCrossEntropy, self).__init__()
        self.class_weight = class_weight
        self.eps = 10e-07 
    
    def call(self, y_true, y_pred):
        """
        y_true and y_pred must be flattened
        """
        # Clipping y_pred for numerical stability 
        y_pred = tf.clip_by_value(y_pred, self.eps, 1-self.eps)

        loss_for_true = tf.negative(tf.multiply(y_true, tf.math.log(y_pred)) * self.class_weight[1])
        loss_for_false = tf.negative(tf.multiply(1. - y_true, tf.math.log(1.0 - y_pred)) * self.class_weight[0])
        weighted_loss_sum = tf.add(loss_for_true, loss_for_false)
        
        return tf.reduce_mean(weighted_loss_sum, axis=0)

In [25]:
WBCE = WeightedBinaryCrossEntropy([0.13, 0.87])

In [26]:
WBCE(y_true=y_flatten, y_pred=y_hat_flatten)

<tf.Tensor: shape=(), dtype=float32, numpy=0.06405591>